# Deploy Agent as Lambda Function

Strands agents agents can be deployed where you would host a Python application, including Amazon EC2, Amazon EKS, and AWS Fargate. You can also deploy many agents to a serverless AWS Lambda function. In this example, we'll package the code and dependencies for our research agent, deploy it to AWS Lambda, and then test the resulting function.

## Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude 3.7 Sonnet model in Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
import boto3
import json
import os
from pathlib import Path
import time

FUNCTION_NAME = "strands-research-agent"
ROLE_NAME = "strands-agent-lambda-role"
script_dir = Path(os.path.abspath(""))
source_dir = script_dir
ZIP_FILE = script_dir / f"{FUNCTION_NAME}.zip"

boto_session = boto3.session.Session()
region = boto_session.region_name
sts_client = boto_session.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
DEPLOYMENT_BUCKET = f"lambda-deployments-{account_id}-{region}"
print(DEPLOYMENT_BUCKET)

## Review Lambda Handler

Let's take a look at the lambda handler function. This is a pretty simple file that defines the agent system prompt and imports the necessary tools.

In [ ]:
%pycat lambda/agent_handler.py

## Prepare Deployment Packages

Next, we package the agent code and dependencies into zip archives and upload them to an Amazon S3 bucket.

In [ ]:
%%sh
pip install -r requirements.txt --python-version 3.12 --platform manylinux2014_aarch64 --target ./packaging/_dependencies --only-binary=:all:
python package_for_lambda.py

In [ ]:
s3_client = boto_session.client("s3")

# Create S3 bucket if it doesn't exist
try:
    s3_client.head_bucket(Bucket=DEPLOYMENT_BUCKET)
    print(f"Using existing S3 bucket: {DEPLOYMENT_BUCKET}")
except:
    print(f"Creating S3 bucket: {DEPLOYMENT_BUCKET}")
    s3_client.create_bucket(Bucket=DEPLOYMENT_BUCKET)

s3_key = f"{FUNCTION_NAME}/{ZIP_FILE.name}"
s3_client.upload_file(
    "packaging/app.zip", DEPLOYMENT_BUCKET, f"{FUNCTION_NAME}/app.zip"
)
s3_client.upload_file(
    "packaging/dependencies.zip", DEPLOYMENT_BUCKET, f"{FUNCTION_NAME}/dependencies.zip"
)

## Create Lambda Execution Role

Next, we create an execution role for our Lambda function to make a call to the Amazon Bedrock API.

In [ ]:
iam_client = boto3.client("iam")

trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole",
        }
    ],
}

try:
    # Try to get existing role
    response = iam_client.get_role(RoleName=ROLE_NAME)
    role_arn = response["Role"]["Arn"]
    print(f"Using existing IAM role: {role_arn}")

except iam_client.exceptions.NoSuchEntityException:
    # Create new role
    print(f"Creating IAM role: {ROLE_NAME}")
    response = iam_client.create_role(
        RoleName=ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="Execution role for Strands Research Agent Lambda function",
    )

    role_arn = response["Role"]["Arn"]

    # Attach basic execution policy
    iam_client.attach_role_policy(
        RoleName=ROLE_NAME,
        PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
    )

    print(f"Created IAM role: {role_arn}")
    print(
        "Note: You may need to wait a few minutes for the role to propagate before deploying."
    )

## Create Lambda Function

Next, we create a lambda function using the agent code package in S3.

In [ ]:
TIMEOUT = 900
MEMORY_SIZE = 512
lambda_client = boto_session.client("lambda")

try:
    # Try to update existing function
    response = lambda_client.update_function_code(
        FunctionName=FUNCTION_NAME,
        S3Bucket=DEPLOYMENT_BUCKET,
        S3Key=f"{FUNCTION_NAME}/app.zip",
    )
    print(f"Updated existing function: {FUNCTION_NAME}")
    time.sleep(15)

    # Update configuration
    lambda_client.update_function_configuration(
        FunctionName=FUNCTION_NAME,
        Timeout=TIMEOUT,
        MemorySize=MEMORY_SIZE,
    )

except lambda_client.exceptions.ResourceNotFoundException:
    # Create new function
    response = lambda_client.create_function(
        FunctionName=FUNCTION_NAME,
        Runtime="python3.12",
        Role=role_arn,
        Handler="agent_handler.handler",
        Architectures=["arm64"],
        Code={"S3Bucket": DEPLOYMENT_BUCKET, "S3Key": f"{FUNCTION_NAME}/app.zip"},
        Timeout=TIMEOUT,
        MemorySize=MEMORY_SIZE,
    )
    print(f"Created new function: {FUNCTION_NAME}")

## Create Lambda Layer

Finally, we create a Lambda layer with the agent code dependencies and attach it to the function.

In [ ]:
layer_details = lambda_client.publish_layer_version(
    LayerName="strands-research-agent-dependencies",
    Description="Dependencies for the Strands Agents deployment",
    Content={
        "S3Bucket": DEPLOYMENT_BUCKET,
        "S3Key": f"{FUNCTION_NAME}/dependencies.zip",
    },
    CompatibleRuntimes=["python3.12"],
    CompatibleArchitectures=["arm64"],
)

layer_version = layer_details.get("Version")
layer_version_arn = layer_details.get("LayerVersionArn")

In [ ]:
lambda_client.update_function_configuration(
    FunctionName=FUNCTION_NAME, Layers=[layer_version_arn]
)

## Invoke Lambda Function

Let's run a test. Depending on the question, it may take 5-10 for the Lambda function to return a response.

In [ ]:
import botocore

config = botocore.config.Config(read_timeout=900, connect_timeout=30)

lambda_client = boto3.client("lambda", config=config)

response = lambda_client.invoke(
    FunctionName=FUNCTION_NAME,
    Payload=json.dumps({"prompt": "What are recent developments in GLP-1 drugs?"}),
)

result = json.loads(response["Payload"].read().decode())
print(result)

Deploying a Strands agent to AWS Lambda gives us a flexible way to integrate scientific insights into a wide variety of workflows.

## (Optional) Clean Up

Delete Lambda Function

In [ ]:
lambda_client.delete_function(FunctionName=FUNCTION_NAME)

Delete Lambda Layer

In [ ]:
lambda_client.delete_layer_version(
    LayerName='strands-research-agent-dependencies',
    VersionNumber=layer_version
)

Delete S3 objects and bucket

In [ ]:
bucket = boto_session.resource("s3").Bucket(DEPLOYMENT_BUCKET)
bucket.objects.filter().delete()
bucket.delete()